# 1. Load up Excel file

In [1]:
import pandas as pd

resfinder_file = '../../drug-key-update/Resfinder 3.0 drug key 072621.xlsx'

resfinder_excel = pd.ExcelFile(resfinder_file)
sheets_df_map_orig = {n: pd.read_excel(resfinder_excel, sheet_name=n, header=None) for n in resfinder_excel.sheet_names}
sheets_df_map_orig.keys()

dict_keys(['Aminoglycoside', 'B-lactam', 'Colistin', 'Fosfomycin', 'Fusidic acid', 'Glycopeptide', 'Macrolide', 'Nitroimidazole', 'Oxazolidinone', 'Phenicol', 'Quinolone', 'Rifampicin', 'Sulphonamide', 'Tetracycline', 'Trimethoprim', 'Non-functional'])

# 2. Add organism names/column names

In [2]:
sheets_df_map = {n.lower().replace(' ', ''): sheets_df_map_orig[n] for n in sheets_df_map_orig}
sheets_df_map['beta-lactam'] = sheets_df_map['b-lactam']
del sheets_df_map['b-lactam']
sheets_df_map.keys()

sheets_df_map = {n: sheets_df_map[n].rename(
    {0: 'gene_accession', 1: 'Drug'}, axis='columns') for n in sheets_df_map}
for n in sheets_df_map:
    sheets_df_map[n].insert(0, 'Class', n)
    
sheets_df_map['aminoglycoside'].head(3)

,Class,gene_accession,Drug,2
0,aminoglycoside,aac(6')-Ib_2_M23634,gentamicin,NaN
1,aminoglycoside,aac(6')-Ib11_1_AY136758,gentamicin,NaN
2,aminoglycoside,aac(6')-30-aac(6')-Ib'_1_AJ584652,gentamicin,NaN


In [3]:
sheets_list = [sheets_df_map[n] for n in sorted(sheets_df_map.keys())]

resfinder_df_orig = pd.concat(sheets_list).reset_index(drop=True)
resfinder_df_orig.head(3)

,Class,gene_accession,Drug,2,3,4,5
0,aminoglycoside,aac(6')-Ib_2_M23634,gentamicin,NaN,NaN,NaN,NaN
1,aminoglycoside,aac(6')-Ib11_1_AY136758,gentamicin,NaN,NaN,NaN,NaN
2,aminoglycoside,aac(6')-30-aac(6')-Ib'_1_AJ584652,gentamicin,NaN,NaN,NaN,NaN


## 2.1. Join together extra columns and see what they contain

In [4]:
df = resfinder_df_orig.drop(['Class', 'gene_accession', 'Drug'],
                       axis='columns')
ds = pd.Series(df.fillna('').values.tolist()).str.join('')

resfinder_df_orig2 = resfinder_df_orig.copy()
resfinder_df_orig2['Other'] = ds
resfinder_df_orig2['Other'].value_counts()

                                                                                                                                                                                                                            3069
formerly aadB                                                                                                                                                                                                                 19
formerly aadE                                                                                                                                                                                                                  2
formerly strA                                                                                                                                                                                                                  2
AMRfinderShig/Ecoeffluxchromosomal wildtype pump                                                    

# 3. Fix up columns

In [5]:
resfinder_df_orig2['gene_accession']

0                     aac(6')-Ib_2_M23634
1                 aac(6')-Ib11_1_AY136758
2       aac(6')-30-aac(6')-Ib'_1_AJ584652
3                  aac(6')-Iaj_1_AB709942
4                  aac(6')-Ian_1_AP014611
                      ...                
3107                    dfrA17_6_AF180469
3108                    dfrA17_7_AB196349
3109                    dfrA17_8_AM932673
3110                    dfrA17_9_FJ807902
3111                   dfrA17_10_AM937244
Name: gene_accession, Length: 3112, dtype: object

In [6]:
# Get rid of all "non-functional" entries
resfinder_df_orig3 = resfinder_df_orig2.copy()
resfinder_df_orig3 = resfinder_df_orig3[resfinder_df_orig3['Class'] != 'non-functional'].copy()

# Split apart the gene and accession
extracted_columns = resfinder_df_orig3['gene_accession'].str.extract(r'^(.*?)_(N[^_]_[^_]+|[^_]+)$',
                                                                     expand=True)
resfinder_df_orig3[['Gene', 'Accession']] = extracted_columns
resfinder_df_orig3['Accession'] = resfinder_df_orig3['Accession'].str.replace(r'^_', '',
                                                                             regex=True)

resfinder_df = resfinder_df_orig3[['Class', 'Gene', 'Accession', 'Drug']].copy()

# Everything should be lower-case
resfinder_df['Drug'] = resfinder_df['Drug'].str.lower()
# Except I/R, which should be upper-case
resfinder_df['Drug'] = resfinder_df['Drug'].str.replace('i/r', 'I/R')

# There are mixtures of spaces and commas separating drugs here. First get rid of commas
resfinder_df['Drug'] = resfinder_df['Drug'].str.replace(',', ' ')

# Get rid of spaces at the very beginning or end
resfinder_df['Drug'] = resfinder_df['Drug'].str.strip()

# Now, get rid of spaces and replace with commas
resfinder_df['Drug'] = resfinder_df['Drug'].str.replace('\s+', ',', regex=True)

# Now fix up specific cases where there should be spaces
resfinder_df['Drug'] = resfinder_df['Drug'].str.replace(',acid', ' acid')
resfinder_df['Drug'] = resfinder_df['Drug'].str.replace(',I/R', ' I/R')
resfinder_df['Drug'] = resfinder_df['Drug'].str.\
  replace('(,decreased,susceptibility,to,meropenem)',
          'decreased susceptibility to meropenem', regex=False)

# Fix up lack of spaces or other issues
resfinder_df['Drug'] = resfinder_df['Drug'].str.replace('inI/R', 'in I/R')
resfinder_df['Drug'] = resfinder_df['Drug'].str.replace('(I/R)', ' I/R', regex=False)

# Change "None" to NA
resfinder_df['Drug'] = resfinder_df['Drug'].replace('None', pd.NA)
resfinder_df['Drug'] = resfinder_df['Drug'].replace('none', pd.NA)

resfinder_df

,Class,Gene,Accession,Drug
0,aminoglycoside,aac(6')-Ib_2,M23634,gentamicin
1,aminoglycoside,aac(6')-Ib11_1,AY136758,gentamicin
2,aminoglycoside,aac(6')-30-aac(6')-Ib'_1,AJ584652,gentamicin
3,aminoglycoside,aac(6')-Iaj_1,AB709942,gentamicin
4,aminoglycoside,aac(6')-Ian_1,AP014611,gentamicin
...,...,...,...,...
3107,trimethoprim,dfrA17_6,AF180469,trimethoprim
3108,trimethoprim,dfrA17_7,AB196349,trimethoprim
3109,trimethoprim,dfrA17_8,AM932673,trimethoprim
3110,trimethoprim,dfrA17_9,FJ807902,trimethoprim


In [7]:
# Look at one of the more problematic cases of extracting the accession number
resfinder_df_orig3[resfinder_df_orig3['Accession'].str.contains('012555')]

,Class,gene_accession,Drug,2,3,4,5,Other,Gene,Accession
151,aminoglycoside,aac(6')-IIc_1_NC_012555,amikacin gentamicin tobramycin,NaN,NaN,NaN,NaN,,aac(6')-IIc_1,NC_012555


# 4. Save ResFinder resistance table

In [8]:
resfinder_df.to_csv('../../staramr/databases/resistance/data/ARG_drug_key_resfinder.tsv', sep='\t', index=False)